In [1]:
LABEL_MAPPING = {
    1: "aorta",
    2: "left_lung",
    3: "right_lung",
    4: "trachea",
    5: "pulmonary_artery",
}
MASK_PATH = "./masks.nii.gz"
SCAN_PATH = "./scan.nii.gz"
CONFIG_PATH = "./extractor_config.json"

In [2]:

 !python3.10 -m pip install -r requirements.txt
!python3.10 setup.py install
!python3.10 setup.py build_ext --inplace
from __future__ import print_function
import sys
import os
import logging
import six


Defaulting to user installation because normal site-packages is not writeable
/home/verscup/.local/lib/python3.10/site-packages/setuptools/config/_apply_pyprojecttoml.py:79: SetuptoolsWarning: `install_requires` overwritten in `pyproject.toml` (dependencies)
  corresp(dist, value, root_dir)
running install
/home/verscup/.local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/home/verscup/.local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDepreca

In [3]:
from radiomics.featureextractor import RadiomicsFeatureExtractor
import os
print("Current working directory:", os.getcwd())
os.chdir('radiomics/cuda/biecek_test')
print("Current working directory after change:", os.getcwd())
print("Files in current directory:", os.listdir())

Current working directory: /home/verscup/pyradiomics-CUDA
Current working directory after change: /home/verscup/pyradiomics-CUDA/radiomics/cuda/biecek_test
Files in current directory: ['testImportat.ipynb', 'scan.nii.gz', 'extractor_config.json', 'masks.nii.gz']


In [4]:
extractor = RadiomicsFeatureExtractor()
extractor.loadParams(CONFIG_PATH)

In [5]:
features = {}
for val, name in LABEL_MAPPING.items():
    features[name] = extractor.execute(SCAN_PATH, MASK_PATH, val)